In [ ]:
import pickle
# make sure you have the .pkl file because it doesn't work with the .zip
with open("sampled_SemCore+OMSTI.pkl", "rb") as f:
    data = pickle.load(f)
data_list = list(data.items())

# YOU ONLY NEED TO CHANGE THE WORD INDEX HERE 
target_word_index = 0

print(data_list[target_word_index][0])

# Structure of data_list
# data_list[word index][0-word, 1-all the info][meaning key][sentence No]['sentence'/'lematized'/'word'/'pos'/'id'/'sent_pos']
# 'sentence' = the whole sentence
# 'lematized' = lemmatization of words in sentence
# 'word' = target word
# 'pos' = NOUN/VERB/ADJ...
# 'id' = some id
# 'sent_pos' = position of target word in sentence
keys = list(data_list[target_word_index][1].keys())

In [ ]:
sentences_lem = []  # contains sentence, lemmatization, target word, ... in a dictionary
only_sentence = []  # contains only the sentences
only_lem = []       # contains lists of the lemmatized words in each sentence
for k in keys:
    for i in data_list[target_word_index][1][k]:
        sentences_lem.append(i)
        only_sentence.append(i['sentence'])
        only_lem.append(i['lemmatized'])

only_lem

In [ ]:
len(sentences_lem)

In [ ]:
only_lem[0]

In [ ]:
# lists of lists of the lemmatized words in the sentences
lem_words_in_sentences = []
for i in only_lem:
    sentence = []
    for j in i:
        sentence.append(j[0])
    lem_words_in_sentences.append(sentence)

lem_words_in_sentences

In [ ]:
# the original sentences but the words in them have been lemmatized
lemmatized_sentences = []
for l in lem_words_in_sentences:
    sen = ' '.join(l)
    print(sen)
    lemmatized_sentences.append(sen)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT-base-uncased model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)


In [10]:
# tokenize the list of lemmatized sentences
inputs = tokenizer(text=lemmatized_sentences, return_tensors="pt", padding=True, truncation=True)

In [11]:
# feed the model
with torch.no_grad():
    outputs = model(**inputs)

In [12]:
# get the hidden layers
all_hidden_states = outputs.hidden_states
word_embeddings = all_hidden_states[0]

In [ ]:
word_embeddings

In [ ]:
all_hidden_states[4].shape